### Определение схожести между двумя объектами

In [4]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

# Загрузка предобученной модели ResNet18
model = models.resnet18(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # Удаление последнего слоя классификации
model.eval()

# Преобразования для изображений
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/annapetrov/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:03<00:00, 12.3MB/s]


In [10]:
def extract_features(image_path, model):
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = model(image).squeeze().numpy()
    return features

In [28]:
from scipy.spatial.distance import cosine

def are_images_similar(image_path1, image_path2, model, threshold=0.5):
    features1 = extract_features(image_path1, model)
    features2 = extract_features(image_path2, model)
    similarity = 1 - cosine(features1, features2)
    return similarity > threshold, similarity

# Пример использования
image_path1 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810155852.webp'
image_path2 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810156155.webp'
image_path3 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н8/6836725625.webp'
image_path4 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/2/7057584592.webp'
image_path5 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/2/7057584629.webp'
image_path6 = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/9/6795482421.webp'
similar, similarity_score = are_images_similar(image_path4, image_path6, model)
print(f"Похожи ли изображения: {similar}, Схожесть: {similarity_score:.2f}")

Похожи ли изображения: True, Схожесть: 0.60


### Определение объектов на изображении

In [9]:
import torchvision.transforms as T

# Загрузка модели для детекции объектов
object_detection_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
object_detection_model.eval()

# Преобразования для детекции
detection_transform = T.Compose([
    T.ToTensor(),
])

def detect_objects(image_path, model, threshold=0.5):
    image = Image.open(image_path)
    image_tensor = detection_transform(image)
    with torch.no_grad():
        predictions = model([image_tensor])[0]
    
    detected_objects = []
    for idx, score in enumerate(predictions['scores']):
        if score > threshold:
            label = predictions['labels'][idx].item()
            box = predictions['boxes'][idx].tolist()
            detected_objects.append((label, box, score.item()))
    
    return detected_objects

# Пример использования
image_path = '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810155852.webp'
detected_objects = detect_objects(image_path, object_detection_model)
print(f"Обнаруженные объекты: {detected_objects}")

/Users/annapetrov/Desktop/ozon_хакатон/detect_object/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/annapetrov/Desktop/ozon_хакатон/detect_object/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обнаруженные объекты: []


### Определение схожести между списком объектов

In [17]:
image_paths = ['/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н9/6971608467.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч2/7021439607.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810155850.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/платья/9/6795482421.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч4/6810156140.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/чехлы/ч9/7061090493.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н3/6873617147.webp',
               '/Users/annapetrov/Desktop/ozon_хакатон/hakaton_ozon_opg/dataset/Наушники/н8/6122417752.webp']

In [21]:
from sklearn.decomposition import PCA

def cluster_images(image_paths, model, n_clusters=3):
    features = np.array([extract_features(image_path, model) for image_path in image_paths])
    
    # Для снижения размерности (опционально)
    min_samples = min(len(image_paths), features.shape[1])
    n_components = min(50, min_samples - 1)  # Убедиться, что n_components допустимо
    
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    
    # Кластеризация
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(reduced_features)
    
    return clusters

clusters = cluster_images(image_paths, model, n_clusters=3)
print(f"Кластеры изображений: {clusters}")

Кластеры изображений: [2 1 1 2 0 1 1 2]


In [20]:
def cluster_images_without_pca(image_paths, model, n_clusters=3):
    features = np.array([extract_features(image_path, model) for image_path in image_paths])
    
    # Кластеризация
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(features)
    
    return clusters

clusters = cluster_images_without_pca(image_paths, model, n_clusters=3)
print(f"Кластеры изображений: {clusters}")

Кластеры изображений: [2 1 1 2 1 1 0 2]
